In [1]:
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from scipy.sparse import vstack, csr_matrix
import joblib

In [2]:
path = r'E:\Projetos\llm_security\dataset'
column_names = [
    "duration", "protocol_type", "service", "flag", "src_bytes", "dst_bytes", "land",
    "wrong_fragment", "urgent", "hot", "num_failed_logins", "logged_in",
    "num_compromised", "root_shell", "su_attempted", "num_root", "num_file_creations",
    "num_shells", "num_access_files", "num_outbound_cmds", "is_host_login",
    "is_guest_login", "count", "srv_count", "serror_rate", "srv_serror_rate",
    "rerror_rate", "srv_rerror_rate", "same_srv_rate", "diff_srv_rate", "srv_diff_host_rate",
    "dst_host_count", "dst_host_srv_count", "dst_host_same_srv_rate",
    "dst_host_diff_srv_rate", "dst_host_same_src_port_rate", "dst_host_srv_diff_host_rate",
    "dst_host_serror_rate", "dst_host_srv_serror_rate", "dst_host_rerror_rate",
    "dst_host_srv_rerror_rate", "label", "difficulty_level"
]

train_df = pd.read_csv(f'{path}/Train.txt', delimiter=',', header=None, names=column_names)
test_df = pd.read_csv(f'{path}/Test.txt', delimiter=',', header=None, names=column_names)

In [3]:
# Function to convert dataset flags to Scapy-compatible flags
def convert_flag_to_scapy(flag):
    flag_translation = {
        'REJ': 'R',
        'SF': 'PA',
        'S0': 'S',
        'RSTO': 'R',
        'S1': 'S',
        'S2': 'S',
        'S3': 'S',
        'RSTOS0': 'R',
        'OTH': ''
    }
    return ''.join([flag_translation.get(f, '') for f in flag])

# Apply the flag conversion to the 'flag' column
train_df['flag'] = train_df['flag'].apply(convert_flag_to_scapy)
test_df['flag'] = test_df['flag'].apply(convert_flag_to_scapy)

# Convert the multi-class labels into binary labels: 1 for any attack, 0 for normal
train_df['binary_label'] = (train_df['label'] != 'normal').astype(int)
test_df['binary_label'] = (test_df['label'] != 'normal').astype(int)

In [ ]:
train_df.to_parquet(f'{path}/train_net.parquet')
test_df.to_parquet(f'{path}/test_net.parquet')

In [5]:
train_df = pd.read_parquet(f'{path}/train_net.parquet')
test_df = pd.read_parquet(f'{path}/test_net.parquet')

# Selecting only the necessary features
features_to_use = ['protocol_type', 'service', 'flag', 'src_bytes', 'dst_bytes']
categorical_features = ['protocol_type', 'service', 'flag']
numerical_features = ['src_bytes', 'dst_bytes']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', MinMaxScaler(), numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ],
    remainder='drop'  # This drops the columns that are not explicitly transformed
)

# Apply preprocessing to both training and testing data
X_train = preprocessor.fit_transform(train_df[features_to_use])
X_test = preprocessor.transform(test_df[features_to_use])

# Convert to CSR format for models that require it
X_train_csr = csr_matrix(X_train)
X_test_csr = csr_matrix(X_test)

# Convert to dense format for deep learning models
X_train_dense = X_train_csr.toarray()
X_test_dense = X_test_csr.toarray()

# Labels for training and testing
y_train = train_df['binary_label']
y_test = test_df['binary_label']